## Import libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
%matplotlib inline

## Import the data

In [3]:
houses = pd.read_csv('houses_to_rent.csv')

In [4]:
houses.head()

,Unnamed: 0,city,area,rooms,bathroom,parking spaces,floor,animal,furniture,hoa,rent amount,property tax,fire insurance,total
0,0,1,240,3,3,4,-,acept,furnished,R$0,"R$8,000","R$1,000",R$121,"R$9,121"
1,1,0,64,2,1,1,10,acept,not furnished,R$540,R$820,R$122,R$11,"R$1,493"
2,2,1,443,5,5,4,3,acept,furnished,"R$4,172","R$7,000","R$1,417",R$89,"R$12,680"
3,3,1,73,2,2,1,12,acept,not furnished,R$700,"R$1,250",R$150,R$16,"R$2,116"
4,4,1,19,1,1,0,-,not acept,not furnished,R$0,"R$1,200",R$41,R$16,"R$1,257"


## Clean the data

In [5]:
# replace the R$'s
import re
replace_col = ['total','hoa','rent amount','property tax', 'fire insurance']

#clean chars and convers to int
for replaces in replace_col:
  houses[replaces].replace(to_replace="[^0-9]", value="", regex=True, inplace=True)
  houses[replaces] = pd.to_numeric(houses[replaces])

In [6]:
#convert the "-" from floor to zero
houses['floor'].replace(to_replace="[^0-9]", value=0, regex=True, inplace=True)
houses['floor'] = houses['floor'].astype(int)

In [7]:
#drop the first column
houses.drop('Unnamed: 0',axis = 1,inplace = True)

In [8]:
houses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6080 entries, 0 to 6079
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   city            6080 non-null   int64  
 1   area            6080 non-null   int64  
 2   rooms           6080 non-null   int64  
 3   bathroom        6080 non-null   int64  
 4   parking spaces  6080 non-null   int64  
 5   floor           6080 non-null   int64  
 6   animal          6080 non-null   object 
 7   furniture       6080 non-null   object 
 8   hoa             5878 non-null   float64
 9   rent amount     6080 non-null   int64  
 10  property tax    6053 non-null   float64
 11  fire insurance  6080 non-null   int64  
 12  total           6080 non-null   int64  
dtypes: float64(2), int64(9), object(2)
memory usage: 617.6+ KB


# Missing Values

In [10]:
houses.isnull().mean() * 100

city              0.000000
area              0.000000
rooms             0.000000
bathroom          0.000000
parking spaces    0.000000
floor             0.000000
animal            0.000000
furniture         0.000000
hoa               3.322368
rent amount       0.000000
property tax      0.444079
fire insurance    0.000000
total             0.000000
dtype: float64

## Get Dummies

In [11]:
houses = pd.get_dummies(houses, drop_first=True)

### Multiple Imputations by Chained Equations(MICE)

In [12]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

MICE_imputer = IterativeImputer()
houses.iloc[:, :] = MICE_imputer.fit_transform(houses)

# Clean the data II

In [13]:
## remove duplicate rows
houses = houses.drop_duplicates(keep=False)

In [14]:
## since the area is skewed, let's break the area into groups
houses['area_groups'] =  pd.cut(houses.area, [5,30,60,100,200,400,25000], include_lowest=True)


In [15]:
## since the hoa is skewed, let's break the area into groups
houses['hoa'] = np.where(houses['hoa'] < 0, 0, houses['hoa'])
houses['hoa_groups'] = pd.cut(houses.hoa, [-100,250,700,1200,1500,3500,4000,220000], include_lowest=True)

In [16]:
## since the property tax is skewed, let's break the area into groups
houses['property tax'] = np.where(houses['property tax'] < 0, 0, houses['property tax'])
houses['prop_tax_groups'] = pd.cut(houses['property tax'], [-100,35,147,459,1467,366300], include_lowest=True)

## Scaling

In [17]:
# get dummies again
houses = pd.get_dummies(houses, drop_first=True)

X = houses.drop('total', axis=1).values
y = houses['total'].values

In [18]:
from sklearn.preprocessing import StandardScaler

st_scale = StandardScaler()
X = st_scale.fit_transform(X)

## Train - Test sep


In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = .33, random_state=0)

## Import & Define the Models


In [52]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train,y_train)

LinearRegression()

In [54]:
pred = lr.predict(X_test)
pred

array([2476.22128626, 2143.06825994, 3620.28633418, ..., 7845.30232713,
       9149.37911136, 3775.56483998])

## Predict with Pycaret

In [46]:
from pycaret.regression import *
exp_reg102 = setup(data = houses, target = 'total', session_id=123,
                  normalize = True, transformation = True, transform_target = True, 
                  combine_rare_levels = True, rare_level_threshold = 0.05,
                  remove_multicollinearity = True, multicollinearity_threshold = 0.95) 
                  


Setup Succesfully Completed!


,Description,Value
0,session_id,123
1,Transform Target,True
2,Transform Target Method,box-cox
3,Original Data,"(5755, 28)"
4,Missing Values,False
5,Numeric Features,24
6,Categorical Features,3
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [47]:
compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Gradient Boosting Regressor,315.546400,43625220.304800,3316.499400,0.820400,0.062500,0.026600
1,Extra Trees Regressor,265.455800,44846950.034300,3316.277300,0.816200,0.059300,0.016100
2,Random Forest,277.247600,44903673.475200,3328.554000,0.816200,0.063600,0.018200
3,CatBoost Regressor,259.280300,44088232.181100,3430.659600,0.812100,0.055600,0.014500
4,Support Vector Machine,436.555300,44886515.185600,3446.125300,0.811900,0.081200,0.043200
5,Light Gradient Boosting Machine,308.852700,44392861.846900,3453.036800,0.811200,0.064300,0.019900
6,Extreme Gradient Boosting,329.387300,43619821.887000,3435.172700,0.809400,0.066200,0.028300
7,Huber Regressor,587.352100,44897998.938700,3574.102100,0.805700,0.127800,0.076000
8,Random Sample Consensus,660.723800,45042967.323800,3666.452800,0.798400,0.126100,0.079900
9,TheilSen Regressor,679.451400,45086964.974300,3697.417300,0.796000,0.127400,0.080000


In [49]:
tuned_extra_tree = tune_model('et')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,212.8554,3.107122e+06,1762.7031,0.8923,0.0582,0.0187
1,94.2556,3.730315e+04,193.1402,0.9980,0.0207,0.0147
2,672.3558,1.149863e+08,10723.1653,0.2413,0.1325,0.0191
3,102.7386,4.997626e+04,223.5537,0.9976,0.0253,0.0155
4,129.8522,1.219159e+05,349.1645,0.9947,0.0287,0.0167
5,126.8153,1.732673e+05,416.2539,0.9926,0.0297,0.0156
6,1022.8964,3.301901e+08,18171.1327,0.0687,0.1950,0.0195
7,125.1912,2.665193e+05,516.2551,0.9884,0.0458,0.0174
8,132.1677,2.547412e+05,504.7189,0.9896,0.0298,0.0155
9,104.2024,7.824070e+04,279.7154,0.9961,0.0324,0.0170


In [50]:
plot_model(tuned_extra_tree, plot = 'parameter')


,Parameters
bootstrap,False
ccp_alpha,0
criterion,mse
max_depth,70
max_features,auto
max_leaf_nodes,None
max_samples,None
min_impurity_decrease,0
min_impurity_split,None
min_samples_leaf,2
